https://julien.danjou.info/blog/2016/python-exceptions-guide

1) always inherit from (at least) Exception

Re-raising an error in Python 3 (not supported in 2)

"Exception chaining allows you to map one exception type to another, so that a method can throw exceptions defined at the same abstraction level as the method itself, without discarding important debugging information."





In [38]:
try:
    v = {}['a']
except KeyError as e:
    raise ValueError('failed') from e
    raise ValueError('failed; {}'.format(e))

ValueError: failed

In [107]:
import cv2
import numpy as np

class Cv2ExceptionBase(Exception):
    pass

class ImageEmptyException(Cv2ExceptionBase):
    def __init__(self, img):
        super()
        self.img = img
        
    def __str__(self):
        return "Image of type {}, data type {} and shape {} is empty"\
                    .format(type(self.img), self.img.dtype, self.img.shape)
        
class ContourListEmptyException(Cv2ExceptionBase):
    def __init__(self, contour_list):
        super()
        self.contour_list = contour_list
        
    def __str__(self):
        return "Contour list of type {} and length {} is empty"\
                    .format(type(self.contour_list), len(self.contour_list))

class Cv2ValueError(Cv2ExceptionBase, ValueError):
    pass

class Cv2TypeError(Cv2ExceptionBase, TypeError):
    pass

class ContourTypeError(Cv2TypeError):
    def __init__(self, contour_list):
        super()
        self.contour_list = contour_list
        
    def __str__(self):
        return "expected type '{}', got type '{}' instead"\
                    .format(list, type(self.contour_list))

def extract_contours(silhouette):
    if np.count_nonzero(silhouette) == 0:
        raise ImageEmptyException(silhouette)
    contours_data = cv2.findContours(np.copy(silhouette), mode=cv2.RETR_LIST, method=cv2.CHAIN_APPROX_NONE)
    img, contours, hierarchy = contours_data
    return contours

def get_largest(contour_list):
    if not isinstance(contour_list, list):
        raise ContourTypeError(contour_list)
    if len(contour_list) == 0:
        raise ContourListEmptyException(contour_list)
        
    contour_areas = [cv2.contourArea(contour) for contour in contour_list]
        
    largest_idx = np.argmax(np.array(contour_areas))
    return contour_list[largest_idx]

In [113]:
get_largest(4)

ContourTypeError: expected type '<class 'list'>', got type '<class 'int'>' instead

In [103]:
get_largest(None)

ContourTypeError: expected type '<class 'list'>', got type '<class 'NoneType'>' instead

In [104]:
get_largest([])

ContourListEmptyException: Contour list of type <class 'list'> and length 0 is empty

In [82]:
extract_contours(np.zeros(0))

ImageEmptyException: Image of type <class 'numpy.ndarray'>, data type float64 and shape (0,) is empty